In [ ]:
!git clone https://github.com/thinh-vu/vnstock_market_data

In [ ]:
!pip install git+https://github.com/thinh-vu/vnstock.git@beta

In [ ]:
# A function to get old price data from csv, update with new data and save to csv. Input is the directory of csv files
import glob
from datetime import datetime, timedelta
import os
import pandas as pd
from vnstock import *

price_dir = '/content/vnstock_market_data/historical_price/all_time'
csv_path = os.path.join(price_dir, '*.csv')
csv_ls = glob.glob(csv_path)

issue_ticker_ls = []
# Loop through csv_ls, read as dataframe and append to a list of dataframe
for path in csv_ls:
# for path in csv_ls[:1]:
    price_df = pd.read_csv(path)
    ticker = price_df['ticker'][0]
    print(ticker)
    try:
        # print(f'Evaluating {ticker}')
        price_df['time'] = pd.to_datetime(price_df['time']).dt.date
        # get the latest date from the price_df['time']
        latest_date = price_df['time'].max().strftime('%Y-%m-%d')
        # add one day to the latest_date
        next_date = (price_df['time'].max() + timedelta(days=1)).strftime('%Y-%m-%d')
        yesterday = get_date(1, 'day')
        update_df = stock_historical_data (ticker, start_date=next_date, end_date=yesterday, resolution='1D', type='stock')
        # convert time to datetime object
        update_df['time'] = pd.to_datetime(update_df['time']).dt.date
        update_df['ticker'] = ticker
        # concatenate the update_df to price_df, drop duplicates
        price_df = pd.concat([price_df, update_df]).drop_duplicates(subset=['time'], keep='last').reset_index(drop=True)
        # save to csv file
        price_df.to_csv(path, index=False)
    except:
        # print(f'Cannot evaluate {ticker}')
        issue_ticker_ls.append(ticker)
        continue

In [4]:
# Verify the result
price_df.tail(20)

,time,open,high,low,close,volume,ticker
2147,2023-06-15,21400,21400,21200,21200,1800,VRG
2148,2023-06-16,21200,21200,21200,21200,1400,VRG
2149,2023-06-19,20700,20800,20700,20700,3000,VRG
2150,2023-06-20,20900,20900,20900,20900,200,VRG
2151,2023-06-21,20900,20900,20800,20800,2000,VRG
2152,2023-06-22,21900,21900,21900,21900,100,VRG
2153,2023-06-23,21500,21500,21000,21000,14400,VRG
2154,2023-06-26,21200,21200,21000,21100,3300,VRG
2155,2023-06-27,21000,21000,20800,20900,4300,VRG
2156,2023-06-28,20900,21400,20900,21400,300,VRG


In [22]:
# Verify any random file to check whether we save the updated csv file
# pd.read_csv('/content/vnstock_market_data/historical_price/all_time/NFC.csv')